In [1]:
import sys
sys.path.append('../..')
import pybeamtools

[INFO ] [     MainThread] [00:55:46      logging.py  23] [pybeamtools.utils.logging] Starting shared logging thread on PID 18036
[INFO ] [   log_listener] [00:55:46      logging.py  32] [pybeamtools.utils.logging] Logging listener started on PID 18036
[INFO ] [     MainThread] [00:55:46      logging.py  27] [pybeamtools.utils.logging] Logging setup finished


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def time():
    return time.time()*0.5

t = 0.0
def fixed_time():
    return t

In [4]:
from pybeamtools.sim.core import SimulationEngine, ChannelMap
sim = SimulationEngine(time_fun=fixed_time)
sim.TRACE = True

In [5]:
from pybeamtools.sim.devices import EchoEngineDevice, ProxyEngineDevice, TRIGSPEC
echo = EchoEngineDevice(name='echo', initial_data={'TEST:ECHO:1':5, 'TEST:ECHO:2':10})
echo2 = EchoEngineDevice(name='echo2', initial_data={'TEST:ECHO:3':5, 'TEST:ECHO:4':10})

In [6]:
sim.add_device(echo, period=3.0)
sim._update_device('echo')
sim.add_device(echo2, period=5.0)
sim._update_device('echo2')

[DEBUG] [     MainThread] [00:55:47         core.py 197] [SimulationEngine] Added device (echo) with period (3.0)
[DEBUG] [     MainThread] [00:55:47         core.py 197] [SimulationEngine] Added device (echo2) with period (5.0)


{'TEST:ECHO:3': 5, 'TEST:ECHO:4': 10}

In [7]:
print(sim.channels_map_chain)
print(sim.channels_dep_chain)
print(sim.next_periodic_read_time)

{'TEST:ECHO:1': {}, 'TEST:ECHO:2': {}, 'TEST:ECHO:3': {}, 'TEST:ECHO:4': {}}
{}
{'echo': 0.0, 'echo2': 0.0}


In [8]:
proxy = ProxyEngineDevice(name='proxy', channel_map={'TEST:PROXY:1':{'TEST:ECHO:1': TRIGSPEC.PROPAGATE}})
proxy1b = ProxyEngineDevice(name='proxy1b', channel_map={'TEST:PROXY:1B':{'TEST:ECHO:2': TRIGSPEC.IGNORE}})
proxy2 = ProxyEngineDevice(name='proxy2', channel_map={'TEST:PROXY:2':{'TEST:PROXY:1': TRIGSPEC.PROPAGATE}})
proxy2b = ProxyEngineDevice(name='proxy2b', channel_map={'TEST:PROXY:2B':{'TEST:PROXY:1': TRIGSPEC.PROPAGATE}})

In [9]:
sim.add_device(proxy, period=6.0)
sim._update_device('proxy')
sim.add_device(proxy1b, period=7.0)
sim._update_device('proxy1b')
sim.add_device(proxy2, period=9.0)
sim._update_device('proxy2')
sim.add_device(proxy2b, period=11.0)
sim._update_device('proxy2b')

[DEBUG] [     MainThread] [00:55:47         core.py 197] [SimulationEngine] Added device (proxy) with period (6.0)
[DEBUG] [     MainThread] [00:55:47         core.py 197] [SimulationEngine] Added device (proxy1b) with period (7.0)
[DEBUG] [     MainThread] [00:55:47         core.py 197] [SimulationEngine] Added device (proxy2) with period (9.0)
[DEBUG] [     MainThread] [00:55:47         core.py 197] [SimulationEngine] Added device (proxy2b) with period (11.0)


{'TEST:PROXY:2B': 5}

In [10]:
print(sim.channels_map_chain)
print(sim.channels_dep_chain)
print(sim.next_periodic_read_time)

{'TEST:ECHO:1': {}, 'TEST:ECHO:2': {}, 'TEST:ECHO:3': {}, 'TEST:ECHO:4': {}, 'TEST:PROXY:1': {'TEST:ECHO:1': <TRIGSPEC.PROPAGATE: 1>}, 'TEST:PROXY:1B': {'TEST:ECHO:2': <TRIGSPEC.IGNORE: 2>}, 'TEST:PROXY:2': {'TEST:PROXY:1': <TRIGSPEC.PROPAGATE: 1>}, 'TEST:PROXY:2B': {'TEST:PROXY:1': <TRIGSPEC.PROPAGATE: 1>}}
{'TEST:ECHO:1': ['TEST:PROXY:1'], 'TEST:ECHO:2': [], 'TEST:PROXY:1': ['TEST:PROXY:2', 'TEST:PROXY:2B']}
{'echo': 0.0, 'echo2': 0.0, 'proxy': 0.0, 'proxy1b': 0.0, 'proxy2': 0.0, 'proxy2b': 0.0}


In [11]:
#t = sim.time()

In [12]:
t += 10
sim._time_step(t)

[DEBUG] [     MainThread] [00:55:47         core.py 340] [SimulationEngine] Advancing sim to (10.0)
[DEBUG] [     MainThread] [00:55:47         core.py 351] [SimulationEngine] Devices to process: [[0.0, 'echo'], [0.0, 'echo2'], [0.0, 'proxy'], [0.0, 'proxy1b'], [0.0, 'proxy2'], [0.0, 'proxy2b']]
[DEBUG] [     MainThread] [00:55:47         core.py 411] [SimulationEngine] UP (echo): result ({'TEST:ECHO:1': 5, 'TEST:ECHO:2': 10})
[DEBUG] [     MainThread] [00:55:47         core.py 411] [SimulationEngine] >UP (proxy): result ({'TEST:PROXY:1': 5})
[DEBUG] [     MainThread] [00:55:47         core.py 411] [SimulationEngine] >>UP (proxy2): result ({'TEST:PROXY:2': 5})
[DEBUG] [     MainThread] [00:55:47         core.py 411] [SimulationEngine] >>UP (proxy2b): result ({'TEST:PROXY:2B': 5})
[DEBUG] [     MainThread] [00:55:47         core.py 351] [SimulationEngine] Devices to process: [[0.0, 'echo2'], [0.0, 'proxy'], [0.0, 'proxy1b'], [0.0, 'proxy2'], [0.0, 'proxy2b'], [3.0, 'echo']]
[DEBUG] [   

In [13]:
sim.time()

10.0

In [14]:
def f(sub, v):
    print(f'Channel input_var_change_callback', sub.name, v)
sub = sim.subscribe_channel('TEST:ECHO:1')
sub.add_callback(f)

[DEBUG] [     MainThread] [00:55:48         core.py 236] [SimulationEngine] Created subscription for channel (TEST:ECHO:1)


In [15]:
t += 10
sim._time_step(t)

[DEBUG] [     MainThread] [00:55:48         core.py 340] [SimulationEngine] Advancing sim to (20.0)
[DEBUG] [     MainThread] [00:55:48         core.py 351] [SimulationEngine] Devices to process: [[11.0, 'proxy2b'], [12.0, 'echo'], [12.0, 'proxy'], [14.0, 'proxy1b'], [15.0, 'echo2'], [18.0, 'proxy2']]
[DEBUG] [     MainThread] [00:55:48         core.py 411] [SimulationEngine] UP (proxy2b): result ({'TEST:PROXY:2B': 5})
[DEBUG] [     MainThread] [00:55:48         core.py 351] [SimulationEngine] Devices to process: [[12.0, 'echo'], [12.0, 'proxy'], [14.0, 'proxy1b'], [15.0, 'echo2'], [18.0, 'proxy2']]
[DEBUG] [     MainThread] [00:55:48         core.py 411] [SimulationEngine] UP (echo): result ({'TEST:ECHO:1': 5, 'TEST:ECHO:2': 10})
[DEBUG] [     MainThread] [00:55:48         core.py 421] [SimulationEngine] UP (echo): subs for (TEST:ECHO:1)=(5)
[DEBUG] [     MainThread] [00:55:48         core.py  65] [ChannelSubscription] Running (1) callbacks for channel (TEST:ECHO:1)
[DEBUG] [     Main

Channel input_var_change_callback TEST:ECHO:1 5
Channel input_var_change_callback TEST:ECHO:1 5
Channel input_var_change_callback TEST:ECHO:1 5


In [16]:
def f(sub, v):
    print(f'Channel input_var_change_callback 2', sub.name, v)
sub = sim.subscribe_channel('TEST:PROXY:2B')
sub.add_callback(f)

[DEBUG] [     MainThread] [00:55:48         core.py 236] [SimulationEngine] Created subscription for channel (TEST:PROXY:2B)


In [17]:
t += 10
sim._time_step(t)

[DEBUG] [     MainThread] [00:55:48         core.py 340] [SimulationEngine] Advancing sim to (30.0)
[DEBUG] [     MainThread] [00:55:48         core.py 351] [SimulationEngine] Devices to process: [[21.0, 'echo'], [21.0, 'proxy1b'], [22.0, 'proxy2b'], [24.0, 'proxy'], [25.0, 'echo2'], [27.0, 'proxy2']]
[DEBUG] [     MainThread] [00:55:48         core.py 411] [SimulationEngine] UP (echo): result ({'TEST:ECHO:1': 5, 'TEST:ECHO:2': 10})
[DEBUG] [     MainThread] [00:55:48         core.py 421] [SimulationEngine] UP (echo): subs for (TEST:ECHO:1)=(5)
[DEBUG] [     MainThread] [00:55:48         core.py  65] [ChannelSubscription] Running (1) callbacks for channel (TEST:ECHO:1)
[DEBUG] [     MainThread] [00:55:48         core.py 411] [SimulationEngine] >UP (proxy): result ({'TEST:PROXY:1': 5})
[DEBUG] [     MainThread] [00:55:48         core.py 411] [SimulationEngine] >>UP (proxy2): result ({'TEST:PROXY:2': 5})
[DEBUG] [     MainThread] [00:55:48         core.py 411] [SimulationEngine] >>UP (pr

Channel input_var_change_callback TEST:ECHO:1 5
Channel input_var_change_callback 2 TEST:PROXY:2B 5
Channel input_var_change_callback 2 TEST:PROXY:2B 5
Channel input_var_change_callback TEST:ECHO:1 5
Channel input_var_change_callback 2 TEST:PROXY:2B 5
Channel input_var_change_callback 2 TEST:PROXY:2B 5
Channel input_var_change_callback TEST:ECHO:1 5
Channel input_var_change_callback 2 TEST:PROXY:2B 5
Channel input_var_change_callback TEST:ECHO:1 5
Channel input_var_change_callback 2 TEST:PROXY:2B 5
Channel input_var_change_callback 2 TEST:PROXY:2B 5


In [18]:
from pybeamtools.sim.devices import MagnetEngineDevice
mag1 = MagnetEngineDevice('magneto','MAG_VAL','MAG_VAL_EXACT', initial_value=0, low=-2, high=2, noise=0.1, resolution=None, model='instant')

In [19]:
sim.add_device(mag1, period=5.0)
sim._update_device('magneto')

[DEBUG] [     MainThread] [00:55:48         core.py 197] [SimulationEngine] Added device (magneto) with period (5.0)


{'MAG_VAL': 0.06485413436802367, 'MAG_VAL_EXACT': 0}

In [20]:
t += 1
sim._time_step(t)

[DEBUG] [     MainThread] [00:55:48         core.py 340] [SimulationEngine] Advancing sim to (31.0)
[DEBUG] [     MainThread] [00:55:48         core.py 351] [SimulationEngine] Devices to process: [[30.0, 'magneto']]
[DEBUG] [     MainThread] [00:55:48         core.py 411] [SimulationEngine] UP (magneto): result ({'MAG_VAL': -0.15219315227896948, 'MAG_VAL_EXACT': 0})
[DEBUG] [     MainThread] [00:55:48         core.py 351] [SimulationEngine] Devices to process: []
[DEBUG] [     MainThread] [00:55:48         core.py 353] [SimulationEngine] Time step done (1 cycles)


In [21]:
print(sim.channels_map_chain)
print(sim.channels_dep_chain)
print(sim.next_periodic_read_time)

{'TEST:ECHO:1': {}, 'TEST:ECHO:2': {}, 'TEST:ECHO:3': {}, 'TEST:ECHO:4': {}, 'TEST:PROXY:1': {'TEST:ECHO:1': <TRIGSPEC.PROPAGATE: 1>}, 'TEST:PROXY:1B': {'TEST:ECHO:2': <TRIGSPEC.IGNORE: 2>}, 'TEST:PROXY:2': {'TEST:PROXY:1': <TRIGSPEC.PROPAGATE: 1>}, 'TEST:PROXY:2B': {'TEST:PROXY:1': <TRIGSPEC.PROPAGATE: 1>}, 'MAG_VAL': {}, 'MAG_VAL_EXACT': {}}
{'TEST:ECHO:1': ['TEST:PROXY:1'], 'TEST:ECHO:2': [], 'TEST:PROXY:1': ['TEST:PROXY:2', 'TEST:PROXY:2B']}
{'echo': 33.0, 'echo2': 35.0, 'proxy': 36.0, 'proxy1b': 35.0, 'proxy2': 36.0, 'proxy2b': 33.0, 'magneto': 35.0}


In [22]:
t += 1
sim._time_step(t)

[DEBUG] [     MainThread] [00:55:49         core.py 340] [SimulationEngine] Advancing sim to (32.0)
[DEBUG] [     MainThread] [00:55:49         core.py 351] [SimulationEngine] Devices to process: []
[DEBUG] [     MainThread] [00:55:49         core.py 353] [SimulationEngine] Time step done (0 cycles)


In [23]:
from pybeamtools.sim.devices import RPNEngineDevice
rpn1 = RPNEngineDevice(name='rpn1', output_name='TEST:RPN', rpn_expression='TEST:PROXY:2B 10 +')

In [24]:
sim.add_device(rpn1, period=1.0)
sim._update_device(rpn1)

[DEBUG] [     MainThread] [00:55:49         core.py 197] [SimulationEngine] Added device (rpn1) with period (1.0)


{'TEST:RPN': 15.0}

In [25]:
t += 1
sim._time_step(t)

[DEBUG] [     MainThread] [00:55:49         core.py 340] [SimulationEngine] Advancing sim to (33.0)
[DEBUG] [     MainThread] [00:55:49         core.py 351] [SimulationEngine] Devices to process: [[32.0, 'rpn1'], [33.0, 'echo'], [33.0, 'proxy2b']]
[DEBUG] [     MainThread] [00:55:49         core.py 411] [SimulationEngine] UP (rpn1): result ({'TEST:RPN': 15.0})
[DEBUG] [     MainThread] [00:55:49         core.py 351] [SimulationEngine] Devices to process: [[33.0, 'echo'], [33.0, 'proxy2b'], [33.0, 'rpn1']]
[DEBUG] [     MainThread] [00:55:49         core.py 411] [SimulationEngine] UP (echo): result ({'TEST:ECHO:1': 5, 'TEST:ECHO:2': 10})
[DEBUG] [     MainThread] [00:55:49         core.py 421] [SimulationEngine] UP (echo): subs for (TEST:ECHO:1)=(5)
[DEBUG] [     MainThread] [00:55:49         core.py  65] [ChannelSubscription] Running (1) callbacks for channel (TEST:ECHO:1)
[DEBUG] [     MainThread] [00:55:49         core.py 411] [SimulationEngine] >UP (proxy): result ({'TEST:PROXY:1': 

Channel input_var_change_callback TEST:ECHO:1 5
Channel input_var_change_callback 2 TEST:PROXY:2B 5
Channel input_var_change_callback 2 TEST:PROXY:2B 5


In [26]:
def f(sub, v):
    print(f'Channel input_var_change_callback rpn', sub.name, v)
sub = sim.subscribe_channel('TEST:RPN')
sub.add_callback(f)

[DEBUG] [     MainThread] [00:55:49         core.py 236] [SimulationEngine] Created subscription for channel (TEST:RPN)


In [27]:
t += 1
sim._time_step(t)

[DEBUG] [     MainThread] [00:55:49         core.py 340] [SimulationEngine] Advancing sim to (34.0)
[DEBUG] [     MainThread] [00:55:49         core.py 351] [SimulationEngine] Devices to process: [[34.0, 'rpn1']]
[DEBUG] [     MainThread] [00:55:49         core.py 411] [SimulationEngine] UP (rpn1): result ({'TEST:RPN': 15.0})
[DEBUG] [     MainThread] [00:55:49         core.py 421] [SimulationEngine] UP (rpn1): subs for (TEST:RPN)=(15.0)
[DEBUG] [     MainThread] [00:55:49         core.py  65] [ChannelSubscription] Running (1) callbacks for channel (TEST:RPN)
[DEBUG] [     MainThread] [00:55:49         core.py 351] [SimulationEngine] Devices to process: []
[DEBUG] [     MainThread] [00:55:49         core.py 353] [SimulationEngine] Time step done (1 cycles)


Channel input_var_change_callback rpn TEST:RPN 15.0


In [28]:
from pybeamtools.sim.devices import Oscillator, ModelEngineDevice
dev_osc = Oscillator(name='osc1', period=60.0, amplitude=1.0, now=t)
mdev_osc = ModelEngineDevice(device=dev_osc)

In [29]:
mdev_osc.channel_map

{'osc1': {}}

In [30]:
sim.add_device(mdev_osc, period=1.0)
sim._update_device(mdev_osc)

[DEBUG] [     MainThread] [00:55:50         core.py 197] [SimulationEngine] Added device (osc1) with period (1.0)


{'osc1': 0.0}

In [31]:
t += 1
sim._time_step(t)

[DEBUG] [     MainThread] [00:55:50         core.py 340] [SimulationEngine] Advancing sim to (35.0)
[DEBUG] [     MainThread] [00:55:50         core.py 351] [SimulationEngine] Devices to process: [[34.0, 'osc1'], [35.0, 'echo2'], [35.0, 'proxy1b'], [35.0, 'magneto'], [35.0, 'rpn1']]
[DEBUG] [     MainThread] [00:55:50         core.py 411] [SimulationEngine] UP (osc1): result ({'osc1': 0.10452846326765346})
[DEBUG] [     MainThread] [00:55:50         core.py 351] [SimulationEngine] Devices to process: [[35.0, 'echo2'], [35.0, 'proxy1b'], [35.0, 'magneto'], [35.0, 'rpn1'], [35.0, 'osc1']]
[DEBUG] [     MainThread] [00:55:50         core.py 411] [SimulationEngine] UP (echo2): result ({'TEST:ECHO:3': 5, 'TEST:ECHO:4': 10})
[DEBUG] [     MainThread] [00:55:50         core.py 351] [SimulationEngine] Devices to process: [[35.0, 'proxy1b'], [35.0, 'magneto'], [35.0, 'rpn1'], [35.0, 'osc1']]
[DEBUG] [     MainThread] [00:55:50         core.py 411] [SimulationEngine] UP (proxy1b): result ({'TEST

Channel input_var_change_callback rpn TEST:RPN 15.0


In [32]:
sim.latest_data

{'TEST:ECHO:1': 5,
 'TEST:ECHO:2': 10,
 'TEST:ECHO:3': 5,
 'TEST:ECHO:4': 10,
 'TEST:PROXY:1': 5,
 'TEST:PROXY:1B': 10,
 'TEST:PROXY:2': 5,
 'TEST:PROXY:2B': 5,
 'MAG_VAL': 0.23650290165395857,
 'MAG_VAL_EXACT': 0,
 'TEST:RPN': 15.0,
 'osc1': 0.10452846326765346}

In [33]:
sim.latest_data_timestamp

{'TEST:ECHO:1': 33.0,
 'TEST:ECHO:2': 33.0,
 'TEST:PROXY:1': 33.0,
 'TEST:PROXY:2': 33.0,
 'TEST:PROXY:2B': 33.0,
 'TEST:ECHO:3': 35.0,
 'TEST:ECHO:4': 35.0,
 'TEST:PROXY:1B': 35.0,
 'MAG_VAL': 35.0,
 'MAG_VAL_EXACT': 35.0,
 'TEST:RPN': 35.0,
 'osc1': 35.0}

In [ ]:
sim.start_update_thread()
print()

In [ ]:
sim.stop_update_thread()